## BERT Embeddings Serverless Function
This notebook presents deployment of pretrained BERT model that outputs embeddings for given textual sequences as a serverless function. Embeddings are meaningful, contextual representations of text in the form of ndarrays that are used frequently as input to various learning tasks in the field of NLP.

### Example

In [1]:
import nuclio
from nuclio import build_file
import json
import pickle
from test import init_context, handler

In [2]:
context = nuclio.Context()
event = nuclio.Event(body=json.dumps(['John loves Mary']))
init_context(context)
outputs = pickle.loads(handler(context, event))

/User/.pythonlibs/jupyter-alonm/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


This is a good chance to view the outputs of this BERT model. It gives two different outputs. The first is a contextual embedding for each token in the input sequence and the second is a pooled embedding for the complete sequence.

In [3]:
print(f'embeddings per token shape: {outputs[0].shape}, pooled embeddings shape: {outputs[1].shape}')

embeddings per token shape: (1, 5, 768), pooled embeddings shape: (1, 768)


As seen both outputs share first dimension size of 1. This corresponds to the single sequence we passed as input, "John loves Mary". The last dimension for both is of size 768 which is the embedding dimension for this default configuration of bert. Note that the first input has an intermediate dimension of size 5 that corresponds to the number of tokens in the input sequence after addtion of two special tokens marking beginning and end of a sequence by the tokenizer.

### Deploy as serverless function

In [4]:
from mlrun import code_to_function
fn = code_to_function("bert_embeddings", kind="nuclio",
                      description="Get BERT based embeddings for given text",
                      categories=["NLP", "BERT", "embeddings"],
                      labels = {"author": "roye", "framework": "pytorch"},
                      code_output='.')

fn.export("function.yaml")

> 2021-02-15 08:05:23,190 [warning] Unable to parse server or client version. Assuming compatible: {'server_version': 'X', 'client_version': '0.6.0'}
> 2021-02-15 08:05:26,583 [info] function spec saved to path: function.yaml


In [5]:
addr = fn.deploy(project='nlp-servers')

> 2021-02-15 08:05:26,595 [info] Starting remote function deploy
2021-02-15 08:05:26  (info) Deploying function
2021-02-15 08:05:26  (info) Building
2021-02-15 08:05:26  (info) Staging files and preparing base images
2021-02-15 08:05:26  (info) Building processor image
2021-02-15 08:05:27  (info) Build complete
Failed to deploy. Details:
Caught unhandled exception while initializing [err="No module named 'nuclio'" || traceback="Traceback (most recent call last):
  File "/opt/nuclio/_nuclio_wrapper.py", line 350, in run_wrapper
    args.trigger_name)
  File "/opt/nuclio/_nuclio_wrapper.py", line 57, in __init__
    self._entrypoint = self._load_entrypoint_from_handler(handler)
  File "/opt/nuclio/_nuclio_wrapper.py", line 159, in _load_entrypoint_from_handler
    module = __import__(module_name)
  File "/opt/nuclio/bert_embeddings.py", line 3, in <module>
    import nuclio
ModuleNotFoundError: No module named 'nuclio'
" || worker_id="0"]
> 2021-02-15 08:05:32,072 [error] Nuclio function

RunError: cannot deploy Failed to deploy. Details:
Caught unhandled exception while initializing [err="No module named 'nuclio'" || traceback="Traceback (most recent call last):
  File "/opt/nuclio/_nuclio_wrapper.py", line 350, in run_wrapper
    args.trigger_name)
  File "/opt/nuclio/_nuclio_wrapper.py", line 57, in __init__
    self._entrypoint = self._load_entrypoint_from_handler(handler)
  File "/opt/nuclio/_nuclio_wrapper.py", line 159, in _load_entrypoint_from_handler
    module = __import__(module_name)
  File "/opt/nuclio/bert_embeddings.py", line 3, in <module>
    import nuclio
ModuleNotFoundError: No module named 'nuclio'
" || worker_id="0"]

#### Test the function via http request

In [ ]:
import requests


event_data = ['the quick brown fox jumps over the lazy dog', 'Hello I am Jacob']
resp = requests.post(addr, json=json.dumps(event_data))

In [ ]:
output_embeddings = pickle.loads(resp.content)

In [ ]:
print(f'embeddings per token shape: {output_embeddings[0].shape}, pooled embeddings shape: {output_embeddings[1].shape}')

Now we can see that the size of the first dimension of the outputs is two since we passed in two sequences. Also the intermediate dimension of the first output is the maximal number of tokens across all input sequences. Sequences with less tokens are padded with zero values.